In [4]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
colunas = ["PAV", "AMBIENTE", "ÁREA", "TIPO", "ALTURA", "PÉ DIREITO ÚTIL", "REFLETÂNCIA", "FATOR DE DEPRECIAÇÃO", "NÍVEL DE ILUMINÂNCIA (LUX)", "FLUXO LUMINOSO TOTAL (LUMENS)", "COEFICIENTE DE UTILIZAÇÃO", "LÂMPADAS POR LUMINÁRIA", "FLUXO POR LÂMPADAS", "FLUXO POR LUMINÁRIA", "CÁLCULO DE LUMINÁRIAS", "TOTAL DE LUMINÁRIAS", "TOTAL DE LÂMPADAS", "POTÊNCIA UNITÁRIA", "MODELO DE LUMINÁRIA", "TEMPERATURA DA LÂMPADA", "POTENCIA TOTAL", "P. LIMITE A", "ETIQUETA FUTURA"]
df = pd.read_excel("PLANILHA_JF.xlsx", sheet_name=0, names=colunas)


In [6]:
df = df[(df["ALTURA"].notnull()) & (df["ALTURA"] != "PLANTA CAD")].reset_index(drop=True).iloc[1:, :] # REMOVE LINHAS QUE NÃO TEM ALTURA e Remove a primeira linha que é parte do cabeçalho

In [7]:
dfLista = df.values.tolist()  # Transforma o dataframe em lista
dfLista.pop(0)  # Remove o cabeçalho

# Filtra a lista para remover campos onde não há altura da sala
dfListaAntiga = list(filter(lambda x: True 
if (x[6] != "PLANTA CAD" 
and str(x[3]) != "nan" and str(x[3]) != "-" 
and str(x[4]) != "nan" and str(x[4]) != "-" and str(x[4]) != "10,87/9"
and str(x[5]) != "nan" and str(x[5]) != "-" 
and str(x[6]) != "nan" and str(x[6]) != "-" 
and str(x[0]) != "nan" and str(x[1]) != "nan") 
else False, dfLista))

dfListaNova = []

print(df.columns)
for x in dfListaAntiga:
    print(x)
    dfListaNova.append(x)


Index(['PAV', 'AMBIENTE', 'ÁREA', 'TIPO', 'ALTURA', 'PÉ DIREITO ÚTIL',
       'REFLETÂNCIA', 'FATOR DE DEPRECIAÇÃO', 'NÍVEL DE ILUMINÂNCIA (LUX)',
       'FLUXO LUMINOSO TOTAL (LUMENS)', 'COEFICIENTE DE UTILIZAÇÃO',
       'LÂMPADAS POR LUMINÁRIA', 'FLUXO POR LÂMPADAS', 'FLUXO POR LUMINÁRIA',
       'CÁLCULO DE LUMINÁRIAS', 'TOTAL DE LUMINÁRIAS', 'TOTAL DE LÂMPADAS',
       'POTÊNCIA UNITÁRIA', 'MODELO DE LUMINÁRIA', 'TEMPERATURA DA LÂMPADA',
       'POTENCIA TOTAL', 'P. LIMITE A', 'ETIQUETA FUTURA'],
      dtype='object')
['1º', 'Sala de conciliação 3', 14.3, 'Escritório', 2.56, 1.61, 732, 0.7, 500, 18571.428571428572, 0.55, 2, 900, 1800, 10.317460317460318, 10.0, 20, 18.0, 'CAA01-E216', 4000.0, 180, nan, 'B OU +']
['1º', 'Sec. Cejuc', 31.47, 'Escritório', 2.56, 1.61, 732, 0.7, 500, 40870.129870129866, 0.55, 2, 900, 1800, 22.705627705627702, 22.0, 44, 18.0, 'CAA01-E216', 4000.0, 396, nan, 'B OU +']
['1º', 'wc sec. Cejuc', 3.84, 'Banheiro', 2.56, 1.61, 732, 0.7, 200, 1994.8051948051948

In [8]:
dfNovo = pd.DataFrame(dfListaNova, columns=colunas)
dfNovo

,PAV,AMBIENTE,ÁREA,TIPO,ALTURA,PÉ DIREITO ÚTIL,REFLETÂNCIA,FATOR DE DEPRECIAÇÃO,NÍVEL DE ILUMINÂNCIA (LUX),FLUXO LUMINOSO TOTAL (LUMENS),COEFICIENTE DE UTILIZAÇÃO,LÂMPADAS POR LUMINÁRIA,FLUXO POR LÂMPADAS,FLUXO POR LUMINÁRIA,CÁLCULO DE LUMINÁRIAS,TOTAL DE LUMINÁRIAS,TOTAL DE LÂMPADAS,POTÊNCIA UNITÁRIA,MODELO DE LUMINÁRIA,TEMPERATURA DA LÂMPADA,POTENCIA TOTAL,P. LIMITE A,ETIQUETA FUTURA
0,1º,Sala de conciliação 3,14.3,Escritório,2.56,1.61,732,0.7,500,18571.428571,0.55,2,900,1800,10.317460,10.0,20,18.0,CAA01-E216,4000.0,180,NaN,B OU +
1,1º,Sec. Cejuc,31.47,Escritório,2.56,1.61,732,0.7,500,40870.129870,0.55,2,900,1800,22.705628,22.0,44,18.0,CAA01-E216,4000.0,396,NaN,B OU +
2,1º,wc sec. Cejuc,3.84,Banheiro,2.56,1.61,732,0.7,200,1994.805195,0.55,2,900,1800,1.108225,1.0,2,18.0,CAA01-E216,4000.0,18,NaN,B OU +
3,1º,Sala reunião Cejuc,45.38,Escritório,2.56,1.61,732,0.7,500,58935.064935,0.55,2,900,1800,32.741703,32.0,64,18.0,CAA01-E216,4000.0,576,NaN,B OU +
4,1º,Sala de recepção,32.76,Sala de espera,2.56,1.61,732,0.7,300,25527.272727,0.55,2,900,1800,14.181818,14.0,28,18.0,CAA01-E216,4000.0,252,NaN,B OU +
5,1º,Circulação Cejuc,13.4,Circulação,2.56,1.61,732,0.7,100,3480.519481,0.55,2,900,1800,1.933622,2.0,4,18.0,CAA01-E216,4000.0,36,NaN,B OU +
6,1º,Entrada cejuc,26.43,Circulação,2.56,1.61,732,0.7,100,6864.935065,0.55,2,900,1800,3.813853,4.0,8,18.0,CAA01-E216,4000.0,72,NaN,B OU +
7,1º,Cozinha,74.96,Cozinha,2.56,1.61,732,0.7,500,97350.649351,0.55,2,1850,3700,26.310986,26.0,52,36.0,CAA01-E232,4000.0,936,NaN,B OU +
8,1º,Restaurante,63.26,Restaurante,2.56,1.61,732,0.7,200,32862.337662,0.55,2,900,1800,18.256854,18.0,36,18.0,CAA01-E216,4000.0,324,NaN,B OU +
9,1º,Recepção sbes,19.66,Circulação,2.56,1.61,732,0.7,100,5106.493506,0.55,2,900,1800,2.836941,2.0,4,18.0,CAA01-E216,4000.0,36,NaN,B OU +


In [9]:
dfNovo = dfNovo[["PAV", "AMBIENTE", "ALTURA"]]

In [10]:
import pandas as pd

# Lê a planilha "PLANILHA_JF.xlsx" e armazena em um DataFrame
df = pd.read_excel("PLANILHA_JF.xlsx")

# Define a lista colunas2 com os nomes das colunas
colunas2 = ["PAV", "AMBIENTE", "ÁREA", "TIPO", "ALTURA", "PÉ DIREITO ÚTIL", "REFLETÂNCIA", "FATOR DE DEPRECIAÇÃO", "NÍVEL DE ILUMINÂNCIA (LUX)", "FLUXO LUMINOSO TOTAL (LUMENS)", "COEFICIENTE DE UTILIZAÇÃO", "LÂMPADAS POR LUMINÁRIA", "FLUXO POR LÂMPADAS", "FLUXO POR LUMINÁRIA", "CÁLCULO DE LUMINÁRIAS", "TOTAL DE LUMINÁRIAS", "TOTAL DE LÂMPADAS", "POTÊNCIA UNITÁRIA", "MODELO DE LUMINÁRIA", "TEMPERATURA DA LÂMPADA", "POTENCIA TOTAL", "P. LIMITE A", "ETIQUETA FUTURA"]

# Define uma função para verificar e modificar os nomes das colunas
def verificar_colunas(colunas2, df):
    # Obtém o número de colunas na planilha
    n_colunas = len(df.columns)

    # Verifica se o número de nomes na lista colunas2 é igual ao número de colunas na planilha
    if len(colunas2) != n_colunas:
        # Percorre os nomes na lista colunas2
        for i, nome in enumerate(colunas2):
            # Verifica se o nome é diferente do nome da coluna correspondente na planilha
            if nome != df.columns[i]:
                # Se for diferente, coloca o nome da coluna na lista colunas2
                colunas2[i] = df.columns[i]
    
        # Imprime a lista colunas2 atualizada
        print("Lista colunas2:", colunas2)
        print("Lista Planilha:", df.columns)
    elif print("Planilhas iguais"):
        return

# Chama a função verificar_colunas com a lista colunas2 e o DataFrame df
verificar_colunas(colunas2, df)


Planilhas iguais


In [11]:
colunas2 = ["PAV", "AMBIENTE", "ÁREA", "TIPO", "ALTURA", "PÉ DIREITO ÚTIL", "REFLETÂNCIA", "FATOR DE DEPRECIAÇÃO", "NÍVEL DE ILUMINÂNCIA (LUX)", "FLUXO LUMINOSO TOTAL (LUMENS)", "COEFICIENTE DE UTILIZAÇÃO", "LÂMPADAS POR LUMINÁRIA", "FLUXO POR LÂMPADAS", "FLUXO POR LUMINÁRIA", "CÁLCULO DE LUMINÁRIAS", "TOTAL DE LUMINÁRIAS", "TOTAL DE LÂMPADAS", "POTÊNCIA UNITÁRIA", "MODELO DE LUMINÁRIA", "TEMPERATURA DA LÂMPADA", "POTENCIA TOTAL", "P. LIMITE A", "ETIQUETA FUTURA"]
df2 = pd.read_excel("PLANILHA_JF.xlsx", sheet_name=0, names=colunas2)
df2 = df2[df2["POTÊNCIA UNITÁRIA"].notnull() & (df2["POTÊNCIA UNITÁRIA"]!="SI")].reset_index(drop=True).iloc[1:,:] #remove colunas onde não tem potencia e remove a primeira linha que é parte do cabeçalho
df2 = df2[["PAV","AMBIENTE","POTÊNCIA UNITÁRIA","TOTAL DE LÂMPADAS"]]


In [12]:
dfGeral = dfNovo.merge(df2,how="inner", left_on=["PAV","AMBIENTE"], right_on=["PAV","AMBIENTE"])

In [13]:
campos = {"PAV": "pavimento", "AMBIENTE": "ambiente", "COMPRIMENTO": "comprimento", "LARGURA": "largura", "ALTURA": "distanciaPlanoDeTrabalhoTeto", "POT. (W)": "potenciaLampada", "QUANT.": "qntdLampadas"}
dfGeral = dfGeral.rename(columns=campos)
dfGeral.drop_duplicates(inplace=True)
dfGeral

,pavimento,ambiente,distanciaPlanoDeTrabalhoTeto,POTÊNCIA UNITÁRIA,TOTAL DE LÂMPADAS
0,1º,Sala de conciliação 3,2.56,18.0,20
1,1º,Sec. Cejuc,2.56,18.0,44
2,1º,wc sec. Cejuc,2.56,18.0,2
3,1º,Sala reunião Cejuc,2.56,18.0,64
4,1º,Sala de recepção,2.56,18.0,28
5,1º,Circulação Cejuc,2.56,18.0,4
6,1º,Entrada cejuc,2.56,18.0,8
7,1º,Cozinha,2.56,36.0,52
8,1º,Restaurante,2.56,18.0,36
9,1º,Recepção sbes,2.56,18.0,4


### Geração do PDF de Qr-Code

In [14]:
import requests
from fpdf import FPDF
import os

# save FPDF() class into a variable pdf
pdf = FPDF()

# set style and size of font that you want in the pdf
pdf.set_font("Arial", size=15)

# obter o diretório do trabalho atual
script_dir = os.getcwd()

for index, i in enumerate(dfGeral.values.tolist()):
    pavimento = i[0]
    ambiente = i[1]

    pdf.add_page()  # add a page

    # create a cell
    pdf.cell(w=0, h=100, txt=f"PAV. {pavimento} - {ambiente}", ln=1, align='C')

    response = requests.get(
        f"https://us-central1-eficiencia-energetica---jf.cloudfunctions.net/api/ambientesQrCode?pavimento={pavimento}&ambiente={ambiente}"
    ).json()
    
    qrCodeAmbiente = response["qrCodeAmbiente"]
    response2 = requests.get(qrCodeAmbiente)

    # nome do arquivo de imagem
    imgDir = os.path.join(script_dir, f"qrCodeAmbiente{index}.png")

    # salvar a imagem
    with open(imgDir, "wb") as file:
        file.write(response2.content)

    # BytesIO(response2.content)
    pdf.image(imgDir, x=pdf.w/2-50/2, y=pdf.h/2-50/2, w=50, h=50)

    # remover a imagem do diretório
    os.remove(imgDir)

# nome do arquivo PDF
pdf_output_path = os.path.join(script_dir, "qrCodesAmbientes.pdf")

# salvar o PDF no mesmo diretório do script
pdf.output(pdf_output_path)


''

### Carga no Banco de Dados

In [20]:
import requests
import pandas as pd

# Geração do DataFrame dfGeral
dfGeral = dfNovo.merge(df2, how="inner", left_on=["PAV", "AMBIENTE"], right_on=["PAV", "AMBIENTE"])

campos = {
    "PAV": "pavimento",
    "AMBIENTE": "ambiente",
    "COMPRIMENTO": "comprimento",
    "LARGURA": "largura",
    "ALTURA": "distanciaPlanoDeTrabalhoTeto",
    "POTÊNCIA UNITÁRIA": "potenciaLampada",
    "QUANT.": "qntdLampadas"
}

dfGeral = dfGeral.rename(columns=campos)
dfGeral.drop_duplicates(inplace=True)

# Carga no Banco de Dados
for i in dfGeral.values.tolist():
    # Remover campos nulos antes de enviar para o banco de dados
    json = {key: str(value) for key, value in zip(dfGeral.columns, i) if pd.notnull(value)}

    response = requests.post(f"https://us-central1-eficiencia-energetica---jf.cloudfunctions.net/api/ambientes", json=json)
    print(f"PAV - {i[0]} Ambiente - {i[1]} ====> status:{response.status_code}")


PAV - 1º Ambiente - Sala de conciliação 3 ====> status:500
PAV - 1º Ambiente - Sec. Cejuc ====> status:500
PAV - 1º Ambiente - wc sec. Cejuc ====> status:500
PAV - 1º Ambiente - Sala reunião Cejuc ====> status:500
PAV - 1º Ambiente - Sala de recepção  ====> status:500
PAV - 1º Ambiente - Circulação Cejuc  ====> status:500
PAV - 1º Ambiente - Entrada cejuc  ====> status:500
PAV - 1º Ambiente - Cozinha ====> status:500
PAV - 1º Ambiente - Restaurante  ====> status:500
PAV - 1º Ambiente - Recepção sbes ====> status:500
PAV - 1º Ambiente - Dentista ====> status:500
PAV - 1º Ambiente - Wc dentista ====> status:500
PAV - 1º Ambiente - Médico  ====> status:500
PAV - 1º Ambiente - Wc medico ====> status:500
PAV - 1º Ambiente - Principal sebes  ====> status:500
PAV - 1º Ambiente - Seinf  ====> status:500
PAV - 1º Ambiente - Sala de equipamento  ====> status:500
PAV - 1º Ambiente - Sala de TI ====> status:500
PAV - 1º Ambiente - Central de videoconferência 01 ====> status:500
PAV - 1º Ambiente -